### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subjects

### Check data table across sessions

In [ ]:
import subjects

sessions = subjects.nsd.pf_sess + subjects.sd.pf_sess

subjects.data_table(sessions)


### Pooled creation of sessions metadata

In [ ]:
import subjects
from neuropy.core import Animal

# sessions = subjects.sd.allsess #+ subjects.nsd.allsess
# sessions = subjects.sd.ratUday1 + subjects.nsd.ratVday3
sessions = subjects.mua_sess()
for sess in sessions:
    sub_name = sess.sub_name[-1]
    # print(sess.animal.name,sess.animal.sex)

    if sub_name in ["J", "K", "U", "V"]:
        sex = "male"
    else:
        sex = "female"

    if sub_name in ["J"]:
        probe = "Buzsaki-64chan-8shanks"
        brain_region = "CA1"
    elif sub_name in ["K", "N"]:
        probe = "Diagnostic-biochips-128chan-8shanks"
        brain_region = "CA1"
    elif sub_name in ["S"]:
        probe = ["Cambridge-64chan-6shanks", "Diagnostic-biochips-128chan-8shanks"]
        brain_region = "bilateral CA1"
    elif sub_name in ["R", "U", "V"]:
        probe = [
            "Diagnostic-biochips-128chan-8shanks",
            "Diagnostic-biochips-128chan-8shanks",
        ]
        brain_region = "bilateral CA1"

    name_str = sess.filePrefix.name 
    day = 'Day'+name_str[name_str.find('Day')+3]

    animal = Animal(
        name=sess.sub_name, sex=sex, brain_region=brain_region, probe=probe, day=day
    )
    animal.save(sess.filePrefix.with_suffix(".animal"))


### Number of neurons

In [ ]:
sessions = subjects.mua_sess()

neur_df = []
for s,sess in enumerate(sessions):
    n_type = sess.neurons_stable.neuron_type
    n_pyr = np.count_nonzero(n_type=='pyr')
    n_inter = np.count_nonzero(n_type=='inter')
    neur_df.append(pd.DataFrame(dict(pyr=[n_pyr],inter=[n_inter],grp=sess.tag)))
neur_df = pd.concat(neur_df,ignore_index=True)
neur_df.sum()

### Paradigm durations

In [ ]:
from neuropy.core import Epoch

sessions = subjects.ripple_sess()

epoch_dur = []
for s,sess in enumerate(sessions):
    paradigm = sess.paradigm

    # if sess.tag=='SD':
    #     post = sess.paradigm['post'].flatten()
    #     rs = Epoch.from_array([post[0]+5*3600],[post[0]+5*3600],['rs']) 
    #     paradigm = paradigm + rs 

    epoch_dur.append(paradigm.to_dataframe())


epoch_dur = pd.concat(epoch_dur,ignore_index=True)

epoch_dur.groupby('label').mean()/3600

### POST duration lengths

In [ ]:
import numpy as np
import subjects

sessions = subjects.sd.allsess + subjects.nsd.allsess

In [ ]:
for sub,sess in enumerate(sessions):
    post= sess.paradigm['post'].flatten()
    print(np.diff(post)/3600)

### Number of neurons discarded because of stability

In [ ]:
sessions = subjects.mua_sess()

n_neurons,n_stable_neurons = 0,0
for s, sess in enumerate(sessions):

    neurons = sess.neurons.get_neuron_type(['pyr','inter']) 
    n_neurons += neurons.n_neurons

    stable_neurons = sess.neurons_stable.get_neuron_type(['pyr','inter']) 
    n_stable_neurons += stable_neurons.n_neurons


In [ ]:
good_perc = (n_stable_neurons/n_neurons) * 100

print(good_perc)

### Total artifact duration

In [ ]:
sessions = subjects.mua_sess()

total_duration, artifact_duration = 0,0
for s, sess in enumerate(sessions):

    total_duration += sess.eegfile.duration
    if sess.artifact is not None:
        artifact_duration += sess.artifact.durations.sum()

print((artifact_duration/total_duration)*100)



### Average number of spikes

In [ ]:
sessions = subjects.mua_sess()

n_spikes,n_neurons = 0,0
for s, sess in enumerate(sessions):

    n_spikes +=  np.concatenate(sess.neurons) 
    if sess.artifact is not None:
        artifact_duration += sess.artifact.durations.sum()

print((artifact_duration/total_duration)*100)

